In [21]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [22]:
req = requests.get('https://congress.gov.ph/legisdocs/?v=bills')
req

<Response [200]>

In [23]:
soup = BeautifulSoup(req.text)


In [24]:
# dont include index 0
soup.find_all('div', class_='panel panel-default')[0].find_all('div', class_='panel-heading')

[<div class="panel-heading panel-heading-custom"><strong>Bills with downloadable text : <span class="text-warning">12,789</span></strong></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00001</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00002</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00003</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00004</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00005</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00006</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00007</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00008</span></div>,
 <div class="panel-heading"><span class="text-muted text-muted-size">HB00009</span></div>,
 <div class="panel-heading"><span cl

In [25]:
id_list = []
for id_ in soup.find_all('div', class_='panel panel-default')[0].find_all('div', class_='panel-heading')[1:]:
    id_list.append(id_.find('span').text)
len(id_list)

12789

In [26]:
### current problem at of aug 30, 2021 --- iloc 10062 inputs status in mother bill status column
### isupper() not reliable in getting titles, issues with single quotes marks
details_list = []
# drop_list = []
for i,elem in enumerate(soup.find_all('div', class_='panel panel-default')[0].find_all('div', class_='panel-body')):
    tup = []
    
    if i == 10062:
        print(i, elem.find_all('p')[-1].text)
        print(re.match(r'\bMother Bill Status:\b', elem.find_all('p')[-1].text) is not None)
        
#         print('Short Title:' in elem.find_all('p')[1].text)
        print('------end------')
#     if 'Short Title:' in elem.find_all('p')[1].text:
#         drop_list.append(i)

    for z, p_elem in enumerate(elem.find_all('p')):
#         if i == 12478:
#             print(i, p_elem.text)
#         if 'Short Title:' in p_elem.text:
#             drop_list.append(i)
#             if i in range(1000,2001):
#                 print(i, p_elem)
#                 print('\n')

#         if len(p_elem.text.split(':')) == 2:
#             tup.append(p_elem.text.split(':')[1])
#         elif len(p_elem.text.split(':')) == 3:
#             tup.append(p_elem.text.split(':')[1])
#             tup.append(p_elem.text.split(':')[2])
#         else:
#             tup.append(p_elem.text)
#         if i == 12478:
#             print(p_elem.text, '\n')
#         deets = ['Significance:', 'Date Filed:', 'Date Urgent:',
#                  'Principal Author/s:', 'Date Read:', 'Primary Referral:',
#                  'Mother Bill Status:']
#         bool_list = [(deet in p_elem.text) for deet in deets]
#         if (z==0) and not (any(bool_list)):
#             tup.append(p_elem.text)
        text = p_elem.text.strip()
        if text.isupper():
            tup.append(p_elem.text)
            
        elif 'Significance:' in text:
            tup.append(p_elem.text)
        elif 'Date Filed:' in text:
            tup.append(p_elem.text)
        elif 'Date Urgent:' in text:
            pass
        elif 'Principal Author/s:' in text:
            tup.append(p_elem.text)
        elif 'Date Read:' in text:
            tup.append(p_elem.text)
        elif 'Primary Referral:' in text:
            pass
        
        elif re.match(r'\bMother Bill Status:\b', text) is not None:
            split_bill = re.split(r'\SStatus: ', text)
            tup.append(split_bill[0])
            tup.append(split_bill[1])

            
#         elif 'Status:' in p_elem.text:
#             tup.append(p_elem.text)
        elif re.match(r'\bStatus:', text) is not None:
            tup.append(None)
            tup.append(text)
            
        elif 'Short Title:' in text:
            pass
        else:
            tup.append(None)
        
#     if len(tup) > 8:
#         tup.pop(1)
#     if ('Short Title:' in tup[1]):
#         tup.pop(1)
        
    details_list.append(tup)



10062 Status: Reported out 2021-08-25 (#01163);Submitted by JUSTICE, APPROPRIATIONS
False
------end------


In [27]:
text = "AN ACT PROVIDING SECURITY OF TENURE FOR BARANGAY HEALTH WORKERS, AMENDING FOR THIS PURPOSE SECTION 6 (D) OF REPUBLIC ACT 7883, OTHERWISE KNOWN AS THE ꞌBARANGAY HEALTH WORKERS' BENEFITS AND INCENTIVES ACT OF 1995ꞌ"
text.replace("\'", "", len(text)).replace("\ꞌ", "")
# text

'AN ACT PROVIDING SECURITY OF TENURE FOR BARANGAY HEALTH WORKERS, AMENDING FOR THIS PURPOSE SECTION 6 (D) OF REPUBLIC ACT 7883, OTHERWISE KNOWN AS THE ꞌBARANGAY HEALTH WORKERS BENEFITS AND INCENTIVES ACT OF 1995ꞌ'

In [28]:
'12313(ABC)'.isupper()

True

In [29]:
# we want the index 1 element gone, this is the short title
details_list[10062]

['AN ACT CREATING ONE (1) ADDITIONAL BRANCH OF THE REGIONAL TRIAL COURT IN THE EIGHTH JUDICIAL REGION TO BE STATIONED IN THE CITY OF BAYBAY, PROVINCE OF LEYTE, AND APPROPRIATING FUNDS THEREFOR, AMENDING FOR THE PURPOSE SECTION 14 (I) OF BATAS PAMBANSA BLG. 129, OTHERWISE KNOWN AS "THE JUDICIARY REORGANIZATION ACT OF 1980", AS AMENDED',
 'Significance: NATIONAL',
 'Date Filed: 2021-08-25',
 'Principal Author/s: CARI, CARL NICOLAS C.; VELOSO, VICENTE III "CHING" S.E.; YAP, ERIC G.; REVILLA, STRIKE B.; SINGSON-MEEHAN, KRISTINE; AGABAS, TYRONE D.; MACAPAGAL ARROYO, JUAN MIGUEL; QUIMBO, STELLA LUZ A.; ZARATE, CARLOS ISAGANI T.; ERMITA-BUHAIN, ELENITA MILAGROS "EILEEN"; BAUTISTA-BANDIGAN, LORNA P.; BIAZON, RUFFY B.; CALDERON, PETER JOHN D.; CUA, JUNIE E.; CUARESMA, LUISA LLOREN; DALIPE, MANUEL JOSE "MANNIX" M.; RUIZ DAZA, PAUL; FARIÑAS, RUDYS CESAR I G.; GARCIA, JOSE ENRIQUE "JOET" III S.; GASATAYA, GREG G.; GO, ED CHRISTOPHER S.; GONZAGA, RUWEL PETER S.; GONZALEZ, SANDRO L.; HARESCO, TEODOR

In [30]:
[i for i,e in enumerate(details_list) if len(e) > 8]

[]

In [31]:
[i for i,e in enumerate(details_list) if len(e) < 7]

[0,
 1,
 3,
 5,
 9,
 17,
 19,
 25,
 29,
 39,
 40,
 41,
 46,
 53,
 54,
 55,
 56,
 59,
 67,
 75,
 80,
 83,
 85,
 87,
 88,
 90,
 92,
 97,
 101,
 102,
 105,
 110,
 122,
 138,
 139,
 140,
 141,
 143,
 155,
 156,
 162,
 163,
 165,
 166,
 168,
 169,
 171,
 175,
 176,
 177,
 179,
 180,
 191,
 192,
 194,
 195,
 196,
 198,
 199,
 200,
 201,
 203,
 207,
 208,
 209,
 214,
 217,
 218,
 220,
 224,
 229,
 231,
 235,
 236,
 240,
 246,
 250,
 259,
 261,
 264,
 270,
 271,
 277,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 295,
 296,
 297,
 300,
 301,
 302,
 304,
 312,
 322,
 327,
 334,
 335,
 336,
 337,
 338,
 339,
 341,
 342,
 343,
 345,
 349,
 350,
 351,
 353,
 354,
 355,
 356,
 357,
 367,
 368,
 370,
 375,
 378,
 379,
 383,
 385,
 390,
 392,
 395,
 397,
 398,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 419,
 420,
 422,
 423,
 429,
 430,
 433,
 435,
 436,
 449,
 452,
 455,
 456,
 457,
 459,
 460,
 461,
 463,
 464,
 471,
 477,
 478,
 479,
 485,
 486,
 491,
 494,
 495,
 498,
 499,
 505,
 

In [32]:
# check if it is removed
details_list[10062]

['AN ACT CREATING ONE (1) ADDITIONAL BRANCH OF THE REGIONAL TRIAL COURT IN THE EIGHTH JUDICIAL REGION TO BE STATIONED IN THE CITY OF BAYBAY, PROVINCE OF LEYTE, AND APPROPRIATING FUNDS THEREFOR, AMENDING FOR THE PURPOSE SECTION 14 (I) OF BATAS PAMBANSA BLG. 129, OTHERWISE KNOWN AS "THE JUDICIARY REORGANIZATION ACT OF 1980", AS AMENDED',
 'Significance: NATIONAL',
 'Date Filed: 2021-08-25',
 'Principal Author/s: CARI, CARL NICOLAS C.; VELOSO, VICENTE III "CHING" S.E.; YAP, ERIC G.; REVILLA, STRIKE B.; SINGSON-MEEHAN, KRISTINE; AGABAS, TYRONE D.; MACAPAGAL ARROYO, JUAN MIGUEL; QUIMBO, STELLA LUZ A.; ZARATE, CARLOS ISAGANI T.; ERMITA-BUHAIN, ELENITA MILAGROS "EILEEN"; BAUTISTA-BANDIGAN, LORNA P.; BIAZON, RUFFY B.; CALDERON, PETER JOHN D.; CUA, JUNIE E.; CUARESMA, LUISA LLOREN; DALIPE, MANUEL JOSE "MANNIX" M.; RUIZ DAZA, PAUL; FARIÑAS, RUDYS CESAR I G.; GARCIA, JOSE ENRIQUE "JOET" III S.; GASATAYA, GREG G.; GO, ED CHRISTOPHER S.; GONZAGA, RUWEL PETER S.; GONZALEZ, SANDRO L.; HARESCO, TEODOR

In [33]:
df_new = pd.DataFrame(details_list, columns=['Title', 'significance', 'Date Filed', 'Authors', 'Date Read', 'Mother_Bill_Status', 'Status'])

df_new

,Title,significance,Date Filed,Authors,Date Read,Mother_Bill_Status,Status
0,"AN ACT INSTITUTIONALIZING A NATIONAL VALUES, E...",Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,None
1,AN ACT CREATING THE DEPARTMENT OF OVERSEAS FIL...,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,None
2,AN ACT PROVIDING FOR A NATIONAL PROGRAM TO SUP...,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,Status: Transmitted to the Committee on Approp...
3,AN ACT CREATING THE EMERGENCY RESPONSE DEPARTM...,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,None
4,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,"Status: Pending with the Committee on MICRO, S..."
...,...,...,...,...,...,...,...
12784,RESOLUTION OF BOTH HOUSES PROPOSING AMENDMENTS...,Significance: NATIONAL,Date Filed: 2020-02-18,"Principal Author/s: SALCEDA, JOEY SARTE",Date Read: 2020-02-24,None,Status: Pending with the Committee on CONSTITU...
12785,RESOLUTION OF BOTH HOUSES\r\nPROPOSING AMENDME...,Significance: NATIONAL,Date Filed: 2020-08-04,"Principal Author/s: DEFENSOR, LORENZ R.",Date Read: 2020-08-10,None,Status: Pending with the Committee on CONSTITU...
12786,"MOTU PROPRIO INQUIRY, IN AID OF LEGISLATION, ...",Significance: NATIONAL,Date Filed: 2020-12-15,"Principal Author/s: LOPEZ, MANUEL LUIS T.; ERI...",Date Read: 2020-12-16,None,Status: Adopted on 2021-01-20
12787,"MOTU PROPRIO INQUIRY, IN AID OF LEGISLATION, O...",Significance: NATIONAL,Date Filed: 2020-12-15,"Principal Author/s: LOPEZ, MANUEL LUIS T.; ERI...",Date Read: 2020-12-16,None,Status: Adopted on 2021-01-20


In [34]:
df_new.iloc[10062]

Title                 AN ACT CREATING ONE (1) ADDITIONAL BRANCH OF T...
significance                                     Significance: NATIONAL
Date Filed                                       Date Filed: 2021-08-25
Authors               Principal Author/s: CARI, CARL NICOLAS C.; VEL...
Date Read                                                          None
Mother_Bill_Status    Status: Reported out 2021-08-25 (#01163);Submi...
Status                                                             None
Name: 10062, dtype: object

In [35]:
display(df_new[df_new.Title.str.contains('Significance:') == True])
print(len(df_new[df_new.Title.str.contains('Significance:') == True]))

,Title,significance,Date Filed,Authors,Date Read,Mother_Bill_Status,Status
599,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
600,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
970,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
1205,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
3898,Significance: LOCAL,Principal Author/s:,None,Status:,None,None,None
5420,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
6591,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
7423,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
7461,Significance: NATIONAL,Principal Author/s:,None,Status:,None,None,None
9287,Significance: NATIONAL,Date Filed: 2021-05-01,"Principal Author/s: TAMBUNTING, JOY MYRA S.",Date Read: 2021-05-18,None,Status: Pending with the Committee on BASIC ED...,None


16


In [36]:
# for i, e in enumerate(details_list):
#     if e[0] in 'CALL FOR APPROPRIATE LEGISLATION NEEDED TO ADDRESS THE TAAL VOLCANO DISASTER AND REHABILITATION':
#         print(i)

In [39]:
df_new[df_new.Title.isna()]

,Title,significance,Date Filed,Authors,Date Read,Mother_Bill_Status,Status
4,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: CAYETANO, ALAN PETER ""COMP...",Date Read: 2019-07-23,None,"Status: Pending with the Committee on MICRO, S..."
21,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: ESCUDERO, EVELINA G.",Date Read: 2019-07-23,None,Status: Pending with the Committee on LOCAL GO...
25,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: ESCUDERO, EVELINA G.",Date Read: 2019-07-23,None,None
40,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: BENITEZ, JOSE FRANCISCO ""K...",Date Read: 2019-07-23,None,None
41,None,Significance: NATIONAL,Date Filed: 2019-07-01,"Principal Author/s: BENITEZ, JOSE FRANCISCO ""K...",Date Read: 2019-07-23,None,None
...,...,...,...,...,...,...,...
12774,None,Significance: NATIONAL,Date Filed: 2021-08-03,"Principal Author/s: BROSAS, ARLENE D.",Date Read: 2021-08-24,None,Status: Pending with the Committee on WOMEN AN...
12777,None,Significance: NATIONAL,Date Filed: 2021-08-23,"Principal Author/s: SUANSING, ESTRELLITA B.",Date Read: 2021-08-23,None,Status: Pending with the Committee on RULES si...
12778,None,Significance: NATIONAL,Date Filed: 2021-08-23,"Principal Author/s: CASTRO, FRANCE L.",Date Read: 2021-08-23,None,Status: Pending with the Committee on RULES si...
12779,None,Significance: NATIONAL,Date Filed: 2021-08-23,"Principal Author/s: ELAGO, SARAH JANE I.",Date Read: 2021-08-23,None,Status: Pending with the Committee on RULES si...


In [38]:
# NEW PROBLEM REMOVE 'namely:' and revise splitting of title 
# CHECK ASSIGNMENT OF IDS THEY ARE WRONG
df[(df.significance != 'NATIONAL') & (df.significance != 'LOCAL')]

NameError: name 'df' is not defined